## 1 | Import packages

In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import tkinter
from tkinter.filedialog import askopenfilename, askopenfilenames
from collections import defaultdict
from nptdms import TdmsFile
print("done!")

done!


## 2 | Use tkinter to choose a file and get the path to it

In [2]:
# Choose a file to test the code:
root = tkinter.Tk()
root.attributes('-topmost', True) # Make window appear on top
in_path = askopenfilename() # Open dialogue to select file
print(in_path)
root.destroy() # Close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0] # Get path until folder
file_name = os.path.split(in_path)[1] # Get filename
print(folder_name)
print(file_name)

D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5
D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells
dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5


In [ ]:
def importFile(curated_channel = None):
    """
    `importFile` opens a window to select a file to import.
    Returns path and calls `openFile` to extract data."
    """

    root = tkinter.Tk()
    root.attributes('-topmost', True) # Make window appear on top
    in_path = askopenfilename() # Open dialogue to select file
    root.destroy() # Close the root window

    folder_name = os.path.split(in_path)[0] # Get path until folder
    file_name = os.path.split(in_path)[1] # Get filename
    
    voltage_mV, current_pA, command, ttl, extracted_channels, corrected_trial_keys, channel_list, channels_data_frame, time, dt = openFile(in_path, curated_channel) # Call openFile() function
        
    return voltage_mV, current_pA, command, ttl, extracted_channels, corrected_trial_keys, channel_list, channels_data_frame, time, dt, folder_name, file_name

## 3 | Open a file

In [ ]:
def openFile(in_path, curated_channel = None):
    """
    `openFile` checks whether you are attempting to open a `.tdms` or a `.hdf5` file.
    Extracts the data from selected channels.
    """

    if '.tdms' in in_path:
        extracted_channels, time, dt = openTDMSfile(in_path)
    elif '.hdf5' in in_path:
        extracted_channels, corrected_trial_keys, channel_list, channels_data_frame, time, dt = openHDF5file(in_path, curated_channel = curated_channel)
    
    voltage_mV = extracted_channels[0]
    current_pA = extracted_channels[1]
    command = extracted_channels[2]
    ttl = extracted_channels[3]
    
    return voltage_mV, current_pA, command, ttl, extracted_channels, corrected_trial_keys, channel_list, channels_data_frame, time, dt

### 3.1 | Open an .hdf5 file and extract channels

In [12]:
# Choose a .hdf5 file to test the code
root = tkinter.Tk()
root.attributes('-topmost', True) # Make window appear on top
in_path = askopenfilename() # Open dialogue to select file
print(in_path)
root.destroy() # Close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0] # Get path until folder
file_name = os.path.split(in_path)[1] # Get filename
print(folder_name)
print(file_name)

D:/Dropbox (UCL)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5
D:/Dropbox (UCL)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells
dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5


In [ ]:
def openHDF5file(in_path,
                 channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B'],
                 curated_channel = None):
    """
    Opens the selected `.hdf5` file and extracts sorted data from chosen channels.
    
    :channel_list: list of channels to extract. If empty, defaults to 'Channel A', 'Channel B', 'Output A', 'Output B'.
    :curated_channel: e.g. copy of a 'Channel' where some sweeps/trials have been deleted due to noise or quality.
    """

    # Read hdf5 file:
    hdf5_file = h5py.File(in_path, 'r')
    
    # Define empty dictionary to populate with correctly sorted data:
    data_dict = defaultdict(list)
    # Define empty dictionary to populate with corrected trialKeys:
    key_dict = defaultdict(list)
    
    # Iterate through channels to find trial indices and sort them numerically:
    for channel in hdf5_file.keys():
        
        # Fix hdf5 indexing. Otherwise it sorts sweeps alphabetically (as 1, 10, 11, [...], 2, 21, 22...)
        if 'Channel' in channel:
            # Get keys from hdf5 (i.e. the name of each sweep/trial)
            # These have been sorted alphabetically as in strings: 
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            
            # Convert to integers so you can sort numerically:
            trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
            
            # Get the indices that will sort the array:
            trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))
            
            if curated_channel is not None:
                # Use trial keys from curated_channel to ensure same number of trials are present in all channels.
                trialKeysInHDF5 = list(hdf5_file[curated_channel].keys())
                trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
                trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))
        
        # In the case of 'Output' channels, we need to add an extra step.
        # Whereas trialKeys for "Channel" always start at "1", "Output" starts at random values like "14197".
        elif 'Output' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
            trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))
            # Transform them so they start from 1 and can be compared to the curated_channel keys:
            trialKeysInHDF5_int_from1 = [(x-min(trialKeysInHDF5_int)+1) for x in trialKeysInHDF5_int]
     
            if curated_channel is not None:
                # Compare the trial keys starting from 1 to those from the curated channel.
                # Then use the indices of matching keys to extract only the curated trials to analyse.
                trialKeysInHDF5_curated = list(hdf5_file[curated_channel].keys())
                trialKeysInHDF5_curated_int = [int(x) for x in trialKeysInHDF5_curated]
                trialKeysInHDF5_curated_sorting_indices = list(np.argsort(trialKeysInHDF5_curated_int))
                # Sort the curated integer keys so you can use them in the list.index() step.
                trialKeysInHDF5_curated_int_sorted = sorted(trialKeysInHDF5_curated_int)
                # For the sorted curated trial keys, find the index of the value matching each curated_channel trial.
                # Use this as the sorting indices.
                trialKeysInHDF5_sorting_indices = [trialKeysInHDF5_int_from1.index(trialKeysInHDF5_curated_int_sorted[i]) for i in range(len(trialKeysInHDF5_curated_int_sorted))]

        # 'Sweeps_Analysis' will be a copy of either 'Channel A' or 'Channel B' that has been curated.
        # Should be the same provided as curated_channel, which will be used to subset all the channels.
        # Won't be extracted as would only be a duplication.
        elif 'Sweeps_Analysis' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
            trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))

        # To extract 'Time':
        elif 'Time' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            trialKeysInHDF5_sorting_indices = range(len(trialKeysInHDF5))
        
        # In case there is any other channel in the hdf5 file you haven't accounted for:
        else:
            # Print a warning:
            print(f"Unrecognised {channel}: check function. This channel may not be properly sorted.")
            trialKeysInHDF5 = list(hdf5_file[curated_channel].keys())
            trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
            trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))

        # Once you have the correct indices to obtain sorted trial keys, extract the ordered data:
        for i in range(len(trialKeysInHDF5_sorting_indices)):
            correctedTrialKey = trialKeysInHDF5[trialKeysInHDF5_sorting_indices[i]] 
            data_dict[channel].append(np.array(hdf5_file[channel][correctedTrialKey]))
            key_dict[channel].append(correctedTrialKey)
    
    extracted_channels = []
    corrected_trial_keys = []

    # Keep only the useful channels and their trial keys:
    for channel in channel_list:
        extracted_channels.append(data_dict[channel])
        corrected_trial_keys.append(key_dict[channel])
    
    # Get time and delta_t
    if len(data_dict['Time']) >0:
        time = data_dict['Time']
        dt = np.mean(np.diff(time))
    else:
        dt = 0.04
        time = np.linspace(0, len(data_dict['Channel A'][0])*dt, len(['Channel A'][0]))
    
    # Create data frame of data:
    channels_data_frame = pd.DataFrame(extracted_channels, index = channel_list, columns = corrected_trial_keys[0])

    return extracted_channels, corrected_trial_keys, channel_list, channels_data_frame, time, dt

### 3.2 | Open .tdms file and extract channels

In [2]:
# Choose a .tdms file to test the code
root = tkinter.Tk()
root.attributes('-topmost', True) # Make window appear on top
in_path = askopenfilename() # Open dialogue to select file
print(in_path)
root.destroy() # Close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0] # Get path until folder
file_name = os.path.split(in_path)[1] # Get filename
print(folder_name)
print(file_name)

D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/201215_cell1_1__2__OP_VC_clear_nointerval__1.tdms
D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells
201215_cell1_1__2__OP_VC_clear_nointerval__1.tdms


In [3]:
def openTDMSfile(in_path, channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B']):
    """
    `openTDMSfile` returns a list of arrays, where each is a sweep/trial.
    """
    
    # Load .tdms file
    tdms_file = TdmsFile(in_path) 
    data_dict = defaultdict(list)
    
    # Iterate through channels and extract data from sweeps/trials
    for group in tdms_file.groups():
        i=0
        for sweep in group.channels():
            data_dict[group.name].append(sweep.data)
            i+=1
                       
    # Keep only useful channels
    extracted_channels = []

    for channel in channel_list: 
        extracted_channels.append(data_dict[channel])

    # Get time and delta_t
    time = data_dict['Time'][0]
    dt = np.mean(np.diff(time))
    
    return extracted_channels, time, dt

In [27]:
tdms_file = TdmsFile(in_path) 
data_dict = defaultdict(list)
channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B']
trial_keys = []

for group in tdms_file.groups():
    for sweep in group.channels():
        data_dict[group.name].append(sweep.data)
        if group.name == 'Channel A':
            trial_keys.append(sweep.name)
    #print(data_dict[group.name])

extracted_channels = []

for channel in channel_list: 
    extracted_channels.append(data_dict[channel])
    #print(data_dict[channel])

extracted_channels_data_frame = pd.DataFrame(extracted_channels, index = channel_list, columns = trial_keys)
#extracted_channels_data_frame


[array([-1.32261878, -1.45141614, -1.51581482, ..., -1.16162208,
       -0.93622671, -1.06502406]), array([-0.96842605, -0.96842605, -1.0972234 , ..., -0.87182803,
       -1.06502406, -1.12942274]), array([-1.12942274, -0.90402737, -0.87182803, ..., -0.48543595,
       -0.61423331, -0.38883793]), array([-0.45323661, -0.42103727, -0.32443925, ..., -0.51763529,
       -0.51763529, -0.38883793]), array([-0.51763529, -0.61423331, -0.42103727, ..., -0.61423331,
       -0.38883793, -0.45323661]), array([-0.48543595, -0.32443925, -0.32443925, ..., -0.45323661,
       -0.61423331, -0.54983463]), array([-0.51763529, -0.32443925, -0.48543595, ...,  0.0297535 ,
       -0.16344254, -0.1312432 ]), array([ 0.0297535 , -0.09904386, -0.06684452, ..., -0.03464518,
        0.0297535 , -0.19564188]), array([-0.16344254, -0.26004056,  0.0297535 , ..., -0.06684452,
       -0.1312432 , -0.03464518]), array([ 0.15855086,  0.06195284, -0.03464518, ..., -0.00244584,
       -0.19564188, -0.03464518]), array([ 0

In [29]:
extracted_channels_data_frame

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Channel A,"[-1.3226187796973414, -1.4514161367694889, -1....","[-0.9684260451947914, -0.9684260451947914, -1....","[-1.1294227431604715, -0.9040273657916632, -0....","[-0.4532366064980151, -0.4210372663158741, -0....","[-0.5176352867692278, -0.6142333069434128, -0....","[-0.4854359466491322, -0.32443924558328086, -0...","[-0.5176352867692278, -0.32443924558328086, -0...","[0.02975349949308724, -0.09904386278756756, -0...","[-0.16344254374156758, -0.26004056493971733, 0...","[0.15855086227075516, 0.061952840140902694, -0...","[0.061952840140902694, -0.03464518170935623, 0...","[0.3839462483281158, 0.3195475664420201, 0.383...","[-0.2922399052770176, -0.19564188417199485, -0...","[-0.19564188417199485, -0.03464518170935623, -...","[-0.03464518170935623, -0.19564188417199485, -...","[-0.35663858585850955, -0.0024458411236648672,..."
Channel B,"[-33.189777587654284, -33.189777587654284, -34...","[-13.870174582953112, -14.836154759601623, -13...","[-27.071903423030875, -27.071903423030875, -26...","[-0.6684452226398963, -0.9904386278756757, -0....","[-6.464326469394492, -6.786319869044774, -5.49...","[-4.854359466491322, -1.634425437415676, 1.585...","[-4.854359466491322, -6.1423330694341285, -2.9...","[8.669363664395322, 5.771422947326991, 2.87348...","[-0.9904386278756757, -1.634425437415676, -1.6...","[-2.2784122457137728, -0.024458411236648672, 0...","[-16.768115104545526, -14.51416136769489, -7.4...","[6.415409771163951, 9.957337324518733, 9.95733...","[-0.3464518170935623, -2.2784122457137728, -1....","[-23.52997622421571, -20.632035761040658, -20....","[19.295146509532213, 17.685179389954868, 20.26...","[-20.954029147073257, -19.666055601089187, -21..."
Output A,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
Output B,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [22]:
group_name = tdms_file.groups()[1]
print(group_name)
group_name.name

<TdmsGroup with path /'Output A'>


'Output A'